# Cython
Cython is an optimising static compiler for both the Python programming language and the extended Cython programming language.

It can speed tup the numerical computations in python.

Refer to the book for compiling Cython files to Cython extension. Here we will focus on the properties of Cython.

### Introduction

In [ ]:
import cython
import pandas as pd
%load_ext Cython

In [ ]:
%%cython 
## Cython cell magic.
# To declare the variable, just append cdef before the usual c dataType declaration. 
# For full list: refer to the Cython version of numpy https://github.com/cython/cython/blob/master/Cython/Includes/numpy/__init__.pxd#L325
cdef int i1
cdef long i2 
cdef short i3 = 0
cdef long double i4
i1 = 1
i4 = 3.14

Following is a simple example that demonstrates the speed up from Cython.

In [ ]:
%%cython 
## Cython cell magic
def example():
  cdef int i, j = 0 
  for i in range(0, 1000):
    j+=1
  return j
  
def example_py():
  j = 0
  for i in range(0, 1000):
    j+=1
  return j

In [ ]:
%timeit example()
%timeit example_py()

The slowest run took 37.44 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 66.8 ns per loop
100000 loops, best of 5: 12.2 µs per loop


The speed up is from the fact that Cython converts the for loop into pure C and then machine code, so it no longer need to rely on the slow interpreter to look up the data type of $j$ every single time. 

The reason why the slowest run is so slow is likely because the first time we run this code, it will need to do compiling.

However, if we declare the type of j to be object, then python will need to look up the type, so we won't see the performance improvement. As below

In [ ]:
%%cython
def example_object():
  cdef object i, j = 0
  for i in range(0, 1000):
    j+=1
  return j

In [ ]:
%timeit example_object()

10000 loops, best of 5: 23.7 µs per loop


As you may thought, once the variable is declared, you can not assign value of a different type to it (unless it is broadcastable)

In [ ]:
%%cython
cdef int a = 0
a = 1.0


Error compiling Cython file:
------------------------------------------------------------
...
cdef int a = 0
a = 1.0
   ^
------------------------------------------------------------

/root/.cache/ipython/cython/_cython_magic_82cd978dafde37a8891d4e62b5168ffc.pyx:2:4: Cannot assign type 'double' to 'int'


In [ ]:
%%cython
cdef double a = 0.0
a = 1
cdef int b = 2
a = <double> b # This is how you can cast the type

### Functions
cdef returnType  declares a cython function. The problem is that you can only call this cython function from a cython function or python function in the same cython file. If you want to call it externally, you need to write a python function wrapper.

Book is wrong by stating that cython function can only be called from another cython function. In fact "Within a Cython module, Python functions and C functions can call each other freely, but only Python functions can be called from outside the module by interpreted Python code."

cpdef generates a cdef and a def, the def will call cdef.

only use cdef if you are going to call the function often


In [ ]:
%%cython

def sum_python(int a, int b):
  return a + b

def sum_python_loop():
  for i in range(0, 1000):
    sum_python(i,i)

cdef int sum_cython(int a, int b):
  return a + b

def sum_cython_loop():
  for i in range(0, 1000):
    sum_cython(i,i) # It works

cpdef int sum_cpython(int a, int b):
  return a + b

def sum_cpython_loop():
  for i in range(0, 1000):
    sum_cpython(i,i)

cpdef inline int sum_cpython_inline(int a, int b):
  return a + b # It will compile into an inline function

def sum_cpython_inline_loop():
  for i in range(0, 1000):
    sum_cpython_inline(i,i)

In [ ]:
%timeit sum_python_loop()

10000 loops, best of 5: 76.4 µs per loop


In [ ]:
%timeit sum_cython_loop()

The slowest run took 35.90 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 45.8 ns per loop


In [ ]:
%timeit sum_cpython_loop()

The slowest run took 39.67 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 46.5 ns per loop


In [ ]:
%timeit sum_cpython_inline_loop()

The slowest run took 45.70 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 54.8 ns per loop


### Classes
A simple example of a class is shown below.


In [ ]:
%%cython

cdef class Point:
    cdef double x # only viewable to cython
    cdef readonly double y # only readable, not writable
    cdef public double dist # can be read from python file
    def __init__(self, double x, double y):
        self.x = x
        self.y = y
        self.dist = (self.x**2 + self.y**2) ** 0.5

    cdef void move(self, double dx, double dy):
        self.x = self.x + dx
        self.y = self.y + dy

In [ ]:
p = Point(0.1,0.2)
print(p.y)
print(p.dist)
p.dist = 10
print(p.dist)
p.x

0.2
0.223606797749979
10.0


AttributeError: ignored

### Declarations

To use cython function in another cython file (myfile.pyx), write a definition file in (myfile.pxd) extension, such as:
cdef int max(int a, int b)

in your cython file, import by calling
from myfile import max

### C Array

In [ ]:
%%cython
cdef double a = 3
cdef double *a_pointer # declare the pointer as you do in C
a_pointer = &a # Assign the address to the pointer as you do in C
print(a_pointer[0]) # To grab the value, we use the zero notation.

3.0


In [ ]:
%%cython
cdef double arr[10] # 1-D array declaration
cdef double arr52[5][2] # 2-D array declaration

# Cython is row major just like python. Row major means 
# 0 1
# 2 3
# 4 5
# 6 7
# 8 9
# for 5 x 2 matrix
cpdef iter_col_row():
    cdef double arr[1000][1000]
    cdef i = 0
    cdef j = 0
    for i in range(0, 1000):
      for j in range(0,1000):
        arr[i][j] = 1

cpdef iter_row_col():
    cdef double arr[1000][1000]
    cdef i = 0
    cdef j = 0
    for i in range(0, 1000):
      for j in range(0,1000):
        arr[j][i] = 1

In [ ]:
# However, doesn't seem to matter that much?
%timeit iter_col_row()
%timeit iter_row_col()

100 loops, best of 5: 13.7 ms per loop
100 loops, best of 5: 12.9 ms per loop


In [ ]:
%%cython
from libc.stdio cimport printf
cdef double arr[10] # 1-D array declaration
printf("%p\n", arr) # This won't show here. (Run it in python terminal to see it)
printf("%p\n", &arr[0])

### Cython on Numpy Arrays
using numpy array in python will have operation taking place at interpreter level and using cython's version of numpy, you can directly operate on the memory and thus save the overhead cost

In [ ]:
%%cython
cimport numpy as c_np
import numpy as np

def numpy_bench_py():
  py_arr = np.random.rand(1000)
  cdef int i
  for i in range(1000):
    py_arr[i] += 1


def numpy_bench_c():
  cdef c_np.ndarray[double, ndim=1] c_arr
  c_arr = np.random.rand(1000)
  cdef int i
  for i in range(1000):
    c_arr[i] += 1

In [ ]:
%timeit numpy_bench_c()
%timeit numpy_bench_py()

The slowest run took 10.54 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 10.4 µs per loop
1000 loops, best of 5: 270 µs per loop


for 2D matrix iteration though, I don't see clear benefit.

### Memory Views
It references to the memory area. Note that the book has some errors again on this topic. Cython memory views and numpys can not be mixed together!!

In [ ]:
%%cython
import numpy as np
cdef double[:,:] a # This is how you define a memory view in cython
a_np = np.random.rand(4,3)
a = a_np[2:4, :] # This assigns memory view of the 3rd to 4th rows to a.
b_np = np.zeros(3, dtype='float64')
a[0,:] = b_np  # Book does this. It throws an error becaue it is assigning numpy array to cython memory view

# The correct way is as follows
# cdef double[:] b_cython_mv 
# b_cython_mv = b_np
# a[0,:] = b_cython_mv
# print(a_np)

TypeError: ignored

# Numba

Accelerate Python Functions
Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest.

Numba uses LLBM to compile python function into internal representation which is a platform-agnostic language that can be compiled to machine code.

In [ ]:
import numpy as np
import numba as nb
from numba import jit

In [ ]:
@jit(nopython=True)
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    # assuming square input matrix
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

The nopython=True option requires that the function be fully compiled (so that the Python interpreter calls are completely removed), otherwise an exception is raised. These exceptions usually indicate places in the function that need to be modified in order to achieve better-than-Python performance. We strongly recommend always using nopython=True.

The function has not yet been compiled. To do that, we need to call the function. The first time the function gets called, numba will compile it. (See that timeit shows much longer time in one run vs other runs:

In [ ]:
x = np.arange(100).reshape(10, 10)
%timeit go_fast(x)
%timeit go_fast.py_func(x)

The slowest run took 387794.29 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 1.72 µs per loop
The slowest run took 6.26 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 21.9 µs per loop


# Type Specficalization
nb.jit decorator compiles a specialized version of the function once it encounters a new argument type, and it can store what kind of signatures it has seen.

In [ ]:
go_fast.signatures

[(array(int64, 2d, C),)]

In [ ]:
x = np.random.randn(3,3)
go_fast(x)

array([[ 1.76637398,  2.22570811,  1.67952812],
       [-0.55878708,  1.63811769,  1.4580265 ],
       [ 0.56807631,  0.69671012,  0.66037382]])

In [ ]:
go_fast.signatures

[(array(int64, 2d, C),), (array(float64, 2d, C),)]

You can specify the signatures, but if you do that, then only the specified signatures can be passed into it, or an error will be thrown
Full list of signatures can be found here: https://numba.pydata.org/numba-doc/latest/reference/types.html#signatures

In [ ]:
@jit([nb.int64(nb.int64[:], nb.int64[:]), nb.int32(nb.int32[:], nb.int32[:])])  # pass a list of signatures [returnTypeA(argA1Type...), returnTypeB(argB1Type...)]
def specified_signature(a, b): # Function is compiled to machine code when called the first time
  return a[0] + b[0]

In [ ]:
specified_signature(np.zeros(3, dtype=int), np.zeros(3, dtype=int))

0

In [ ]:
specified_signature(np.zeros(3, dtype=float), np.zeros(3, dtype=float))  # You can not pass a float typed array to this function

TypeError: ignored

### Object mode vs native mode
When numba cannot infer variable types it wil still try and compile the code, reverting to the interpreter when the types can't be determined or when certain operations are unsupported. In Numba, this is called object mode and contrasts with the interpreter-free scenario called native mode.


In [ ]:
# @jit(nopython=True)
# def go_fast(a): # Function is compiled to machine code when called the first time
#     trace = 0.0
#     # assuming square input matrix
#     for i in range(a.shape[0]):   # Numba likes loops
#         trace += np.tanh(a[i, i]) # Numba likes NumPy functions
#     return a + trace              # Numba likes NumPy broadcasting

In [ ]:
go_fast.inspect_types()

go_fast (array(int64, 2d, C),)
--------------------------------------------------------------------------------
# File: <ipython-input-72-5caf1bd24eed>
# --- LINE 1 --- 

@jit(nopython=True)

# --- LINE 2 --- 

def go_fast(a): # Function is compiled to machine code when called the first time

    # --- LINE 3 --- 
    # label 0
    #   a = arg(0, name=a)  :: array(int64, 2d, C)
    #   trace = const(float, 0.0)  :: float64
    #   trace.2 = trace  :: float64
    #   del trace

    trace = 0.0

    # --- LINE 4 --- 

    # assuming square input matrix

    # --- LINE 5 --- 
    #   jump 6
    # label 6
    #   $8load_global.0 = global(range: <class 'range'>)  :: Function(<class 'range'>)
    #   $12load_attr.2 = getattr(value=a, attr=shape)  :: UniTuple(int64 x 2)
    #   $const14.3 = const(int, 0)  :: Literal[int](0)
    #   $16binary_subscr.4 = static_getitem(value=$12load_attr.2, index=0, index_var=$const14.3, fn=<built-in function getitem>)  :: int64
    #   del $const14.3
    #   d

Numba is not a panacea. Numba is bad at str operation at the moment

In [ ]:
@nb.jit
def concatenate(strings):
  result = ''
  for s in strings:
    result += s
  return result

In [ ]:
concatenate(['hello','world'])

/usr/local/lib/python3.7/dist-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'strings' of function 'concatenate'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-108-1b9535d3a10b>", line 2:
@nb.jit
def concatenate(strings):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


'helloworld'

In [ ]:
concatenate(["hello","world"])

'helloworld'

In [ ]:
concatenate.inspect_types()

concatenate (reflected list(unicode_type)<iv=None>,)
--------------------------------------------------------------------------------
# File: <ipython-input-108-1b9535d3a10b>
# --- LINE 1 --- 

@nb.jit

# --- LINE 2 --- 

def concatenate(strings):

  # --- LINE 3 --- 
  # label 0
  #   strings = arg(0, name=strings)  :: reflected list(unicode_type)<iv=None>
  #   result = const(str, )  :: Literal[str]()
  #   result.2 = result  :: unicode_type
  #   del result

  result = ''

  # --- LINE 4 --- 
  #   jump 6
  # label 6
  #   $10get_iter.1 = getiter(value=strings)  :: iter(reflected list(unicode_type)<iv=None>)
  #   del strings
  #   $phi12.0 = $10get_iter.1  :: iter(reflected list(unicode_type)<iv=None>)
  #   del $10get_iter.1
  #   jump 12
  # label 12
  #   $12for_iter.1 = iternext(value=$phi12.0)  :: pair<unicode_type, bool>
  #   $12for_iter.2 = pair_first(value=$12for_iter.1)  :: unicode_type
  #   $12for_iter.3 = pair_second(value=$12for_iter.1)  :: bool
  #   del $12for_iter.

In [ ]:
x = ['helloWorld'] * 1000
%timeit concatenate.py_func(x)
%timeit concatenate(x)

10000 loops, best of 5: 73.8 µs per loop
100 loops, best of 5: 1.99 ms per loop


Numba's performance is pretty bad! It adds some extra overhead.

### Vectorize

In [ ]:
@nb.vectorize
def nbvec_cantor(a, b):
  return int(0.5 * (a+b) * (a+b+1) + b)

@np.vectorize
def npvec_cantor(a, b):
  return int(0.5 * (a+b) * (a+b+1) + b)

@nb.vectorize(["float32(float32, float32)"],target="cuda") # The book is old ("gpu" no longer works AND you need to specify the data type for GPU)
def npvec_cantor_gpu(a, b):
  return int(0.5 * (a+b) * (a+b+1) + b)

In [ ]:
x1 = np.random.randn(10,1)
x2 = 3

In [ ]:
%timeit nbvec_sum(x1, x2)
%timeit npvec_sum(x1, x2)

The slowest run took 28.95 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 1.05 µs per loop
The slowest run took 6.72 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 11.3 µs per loop


In [ ]:
x1 = np.random.randn(10,1).astype(np.float32)
x2 = 3

In [ ]:
%timeit npvec_cantor_gpu(x1, x2)  # no cuda available on collab

In [ ]:
npvec_sum(x1, x2)

array([[ 1.87062423],
       [ 1.58587567],
       [-0.41047252],
       [-0.09804057],
       [ 0.49428968],
       [ 0.69557323],
       [-0.49992856],
       [ 3.28721768],
       [-0.74669731],
       [-0.60814567]])

### Generalized Universal Function
universal function can only be defined on scalar values. What if we want to do it for arrays?

For example, given a (10, 3, 3) tensor a, and a (3,3) matrix b, if we want to find the (10,1) vector, where each entry is the Frobenius norm of diff between the slices of a and b.

Contrary to vectorize() functions, guvectorize() functions don’t return their result value: they take it as an array argument, which must be filled in by the function. This is because the array is actually allocated by NumPy’s dispatch mechanism, which calls into the Numba-generated code.


In [ ]:
a = np.random.rand(10, 3, 3)
b = np.random.rand(3,3)

In [ ]:
@nb.guvectorize(['float64[:,:], float64[:,:], float64'], '(n,n), (n,n) -> ()')
def fro(amat, bmat, res):
  nrow, ncol = amat.shape
  s = 0
  for row in range(0, nrow):
    for col in range(0, ncol):
      s = s + (amat[row][col] - bmat[row][col])**2
  res = s ** 0.5

In [ ]:
fro(a, b)

array([-0.59016573, -0.89853614, -0.12917633,  0.49856216, -1.12614115,
       -0.3746816 ,  3.24888336,  0.58222097, -0.8091322 ,  0.0936532 ])

### JIT Classes

In [ ]:
import numba as nb
from numba.experimental import jitclass

In [ ]:
class Node:
  def __init__(self, value):
    self.next = None
    self.value = value
    
class LinkedList:
  def __init__(self):
    self.head = None
  
  def push_front(self, value):
    if self.head == None:
      self.head = Node(value)
    else:
      new_head = Node(value)
      new_head.next = self.head
      self.head = new_head
  
  def show(self):
    node = self.head
    while node is not None:
      print(node.value)
      node = node.next

In [ ]:
lst = LinkedList()
lst.push_front(3)
lst.push_front(2)
lst.push_front(1)
lst.show()

1
2
3


In [ ]:
node_type = nb.deferred_type()

node_spec = [
    ('next', nb.optional(node_type)),
    ('value', nb.int64)
]

@jitclass(node_spec)
class Node:
  def __init__(self, value):
    self.next = None
    self.value = value

node_type.define(Node.class_type.instance_type)

ll_spec = [
    ('head', nb.optional(Node.class_type.instance_type))
]

@jitclass(ll_spec)
class LinkedList:
  def __init__(self):
    self.head = None
  
  def push_front(self, value):
    if self.head == None:
      self.head = Node(value)
    else:
      new_head = Node(value)
      new_head.next = self.head
      self.head = new_head
  
  def show(self):
    node = self.head
    while node is not None:
      print(node.value)
      node = node.next

@nb.jit
def sum_list(lst):
  result = 0
  node = lst.head
  while node is not None:
    result += node.value
    node = node.next
  return result

lst = LinkedList()
for i in range(0, 1000):
  lst.push_front(i)


In [ ]:
%timeit sum_list(lst)
%timeit sum_list.py_func(lst)

10000 loops, best of 5: 21 µs per loop
1000 loops, best of 5: 304 µs per loop


### Conclusion
Try Numba first, for finer handling, use Cython.
Use these tools only if you have very expensive numerical computations.